# Sequence Analysis with Python

Contact: Veli Mäkinen veli.makinen@helsinki.fi

The following assignments introduce applications of hashing with ```dict()``` primitive of Python. While doing so, a rudimentary introduction to biological sequences is given. 
This framework is then enhanced with probabilities, leading to routines to generate random sequences under some constraints, including a general concept of *Markov-chains*. All these components illustrate the usage of ```dict()```, but at the same time introduce some other computational routines to efficiently deal with probabilities.   
The function ```collections.defaultdict``` can be useful.

Below are some "suggested" imports. Feel free to use and modify these, or not. Generally it's good practice to keep most or all imports in one place. Typically very close to the start of notebooks.

In [2]:
from collections import defaultdict
import itertools

import pandas as pd
import numpy as np
from numpy import random

from bs4 import BeautifulSoup
import re

from math import log2



from pprint import pprint as print


# completed exercises: 1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,18, (19)

# codon file saved as default: "Codon usage table.html"

# Markus Vasemägi

The automated TMC tests do not test cell outputs. These are intended to be evaluated in the peer reviews. So it is still be a good idea to make the outputs as clear and informative as possible.

To keep TMC tests running as well as possible it is recommended to keep global variable assignments in the notebook to a minimum to avoid potential name clashes and confusion. Additionally you should keep all actual code exection in main guards to keep the test running smoothly. If you run [check_sequence.py](https://raw.githubusercontent.com/saskeli/data-analysis-with-python-summer-2019/master/check_outputs.py) in the `part07-e01_sequence_analysis` folder, the script should finish very quickly and optimally produce no output.

If you download data from the internet during execution (codon usage table), the parts where downloading is done should not work if you decide to submit to the tmc server. Local tests should work fine.

## DNA and RNA

A DNA molecule consist, in principle, of a chain of smaller molecules. These smaller molecules have some common basic components (bases) that repeat. For our purposes it is sufficient to know that these bases are nucleotides adenine, cytosine, guanine, and thymine with abbreviations ```A```, ```C```, ```G```, and ```T```. Given a *DNA sequence* e.g. ```ACGATGAGGCTCAT```, one can reverse engineer (with negligible loss of information) the corresponding DNA molecule.

Parts of a DNA molecule can *transcribe* into an RNA molecule. In this process, thymine gets replaced by uracil (```U```). 


1. Write a function ```dna_to_rna``` to convert a given DNA sequence $s$ into an RNA sequence. For the sake of exercise, use ```dict()``` to store the symbol to symbol encoding rules. Create a program to test your function.

In [138]:

def dna_to_rna(seq):
    seq = seq.upper()
    d = {'A':'A', 'C':'C', 'G':'G', 'T':'U'}
    
    return "".join(d.get(base) for base in seq)
    
    
if __name__ == '__main__':
    print(dna_to_rna("AACGTGATTTC"))

'AACGUGAUUUC'


### Idea of solution

For each base (character) in string fetch it's corresponding dictionary value.

### Discussion

fill in 

## Proteins

Like DNA and RNA, protein molecule can be interpreted as a chain of smaller molecules, where the bases are now amino acids. RNA molecule may *translate* into a protein molecule, but instead of base by base, three bases of RNA correspond to one base of protein. That is, RNA sequence is read triplet (called codon) at a time. 

2. Consider the codon to amino acid conversion table in http://htmlpreview.github.io/?https://github.com/csmastersUH/data_analysis_with_python_2020/blob/master/Codon%20usage%20table.html. Write a function ```get_dict``` to read the table into a ```dict()```, such that for each RNA sequence of length 3, say $\texttt{AGU}$, the hash table stores the conversion rule to the corresponding amino acid. You may store the html page to your local src directory,
and parse that file.

In [184]:

def get_dict():
    
    try:
        with open("src/Codon usage table.html") as f:
            soup = BeautifulSoup(f)
    except:
        try:
            with open("Codon usage table.html") as f:
                soup = BeautifulSoup(f)
        except:
            print("Something went wrong")
        
    
    
    regex = r"([A-Z]{3})\s([A-Z]{1}|[*]{1})"  # regex magic!
    
    data = str(soup.pre.text) #.replace("<pre>", "").replace("</pre>", "")
    
    fdata = re.findall(regex, data)
    
    
    d = {}
    for codon, amino in fdata:
        d[codon] = amino
    return d

    
if __name__ == '__main__':
    codon_to_aa = get_dict()
    print(codon_to_aa)
    print(len(codon_to_aa))

{'AAA': 'K',
 'AAC': 'N',
 'AAG': 'K',
 'AAU': 'N',
 'ACA': 'T',
 'ACC': 'T',
 'ACG': 'T',
 'ACU': 'T',
 'AGA': 'R',
 'AGC': 'S',
 'AGG': 'R',
 'AGU': 'S',
 'AUA': 'I',
 'AUC': 'I',
 'AUG': 'M',
 'AUU': 'I',
 'CAA': 'Q',
 'CAC': 'H',
 'CAG': 'Q',
 'CAU': 'H',
 'CCA': 'P',
 'CCC': 'P',
 'CCG': 'P',
 'CCU': 'P',
 'CGA': 'R',
 'CGC': 'R',
 'CGG': 'R',
 'CGU': 'R',
 'CUA': 'L',
 'CUC': 'L',
 'CUG': 'L',
 'CUU': 'L',
 'GAA': 'E',
 'GAC': 'D',
 'GAG': 'E',
 'GAU': 'D',
 'GCA': 'A',
 'GCC': 'A',
 'GCG': 'A',
 'GCU': 'A',
 'GGA': 'G',
 'GGC': 'G',
 'GGG': 'G',
 'GGU': 'G',
 'GUA': 'V',
 'GUC': 'V',
 'GUG': 'V',
 'GUU': 'V',
 'UAA': '*',
 'UAC': 'Y',
 'UAG': '*',
 'UAU': 'Y',
 'UCA': 'S',
 'UCC': 'S',
 'UCG': 'S',
 'UCU': 'S',
 'UGA': '*',
 'UGC': 'C',
 'UGG': 'W',
 'UGU': 'C',
 'UUA': 'L',
 'UUC': 'F',
 'UUG': 'L',
 'UUU': 'F'}
64


### Idea of solution

Parsing html when there's no <'table'> tag turned out to be challenging. BeautifulSoup offers a functionality to filter html text for tags. 

There was still a bit of clean up to do, and the RegEx which leaves out the space in the middle took a while to formulate. Then iterating through the list of tuples was relatively straight forward. 

### Discussion

Realized a bit later that regex matching can be quite powerful, so filtering for the specific "pre" tag wasn't even necessary, oh well.  

3. Use the same conversion table as above, but now write function `get_dict_list` to read the table into a `dict()`, such that for each amino acid the hash table stores the list of codons encoding it.    

In [231]:

def get_dict_list():
    try:
        with open("src/Codon usage table.html") as f:
            soup = BeautifulSoup(f)
    except:
        try:
            with open("Codon usage table.html") as f:
                soup = BeautifulSoup(f)
        except:
            print("Something went wrong")
    
    regex = r"([A-Z]{3})\s([A-Z]{1}|[*]{1})"  # regex magic
    
    data = str(soup.pre.text) 
    fdata = re.findall(regex, data)
    
    #d = {}
    dd = defaultdict(list)

    for codon, amino in fdata:
        dd[amino].append(codon)
        #d.setdefault(amino, []).append(codon)  ## alternative without defaultdit
    return dict(dd)
    
    
if __name__ == '__main__':
    aa_to_codons = get_dict_list()
    
    
    
    print(aa_to_codons)
    #print(len(aa_to_codons))

{'*': ['UAA', 'UGA', 'UAG'],
 'A': ['GCU', 'GCC', 'GCA', 'GCG'],
 'C': ['UGU', 'UGC'],
 'D': ['GAU', 'GAC'],
 'E': ['GAA', 'GAG'],
 'F': ['UUU', 'UUC'],
 'G': ['GGU', 'GGC', 'GGA', 'GGG'],
 'H': ['CAU', 'CAC'],
 'I': ['AUU', 'AUC', 'AUA'],
 'K': ['AAA', 'AAG'],
 'L': ['UUA', 'UUG', 'CUU', 'CUC', 'CUA', 'CUG'],
 'M': ['AUG'],
 'N': ['AAU', 'AAC'],
 'P': ['CCU', 'CCC', 'CCA', 'CCG'],
 'Q': ['CAA', 'CAG'],
 'R': ['CGU', 'CGC', 'CGA', 'CGG', 'AGA', 'AGG'],
 'S': ['UCU', 'UCC', 'UCA', 'UCG', 'AGU', 'AGC'],
 'T': ['ACU', 'ACC', 'ACA', 'ACG'],
 'V': ['GUU', 'GUC', 'GUA', 'GUG'],
 'W': ['UGG'],
 'Y': ['UAU', 'UAC']}


### Idea of solution

With a mapping of one key to many values creating a dictionary can be a challenge. Collections.defaultdict() is useful here. Also provided is a commented option with a traditional dict. 

An aminoacid can be made out of one of several codons. 

### Discussion

fill in 

With the conversion tables at hand, the following should be trivial to solve.

4. Fill in function ```rna_to_prot``` in the stub solution to convert a given DNA sequence $s$ into a protein sequence. 
You may use the dictionaries from exercises 2 and 3. You can test your program with `ATGATATCATCGACGATGTAG`.

In [232]:
#rna = "AUGAUAUCAUCGACGAUGUAG"


def rna_to_prot(seq):
    d = get_dict()
    
    liste = [seq[i: i+3] for i in range(0, len(seq), 3)] # string of codons organized into list
    
    
    return ''.join(d[base][0] for base in liste)      # converting each codon to amino, then back to string

def dna_to_prot(seq):
    return rna_to_prot(dna_to_rna(seq))


if __name__ == '__main__':
    print(dna_to_prot("ATGATATCATCGACGATGTAG"))
    #print(rna_to_prot(rna))

'MISSTM*'


### Idea of solution

Handling and grouping of 3 characters of a string while looping through the string seemed quite complex. I found it easier to convert each group of 3 to an element in a list, and then getting the corresponding amino acid from the previous get_dict() dictionary. Then simply converting back to string format.   

### Discussion

fill in 

You may notice that there are $4^3=64$ different codons, but only 20 amino acids. That is, some triplets encode the same amino acid.  

## Reverse translation

It has been observed that among the codons coding the same amino acid, some are more frequent than others. These frequencies can be converted to probabilities. E.g. consider codons `AUU`, `AUC`, and `AUA` that code for amino acid isoleucine.
If they are observed, say, 36, 47, 17 times, respectively, to code isoleucine in a dataset, the probability that a random such event is `AUU` $\to$ isoleucine is 36/100.

This phenomenon is called *codon adaptation*, and for our purposes it works as a good introduction to generation of random sequences under constraints.   

5. Consider the codon adaptation frequencies in http://htmlpreview.github.io/?https://github.com/csmastersUH/data_analysis_with_python_2020/blob/master/Codon%20usage%20table.html and read them into a ```dict()```, such that for each RNA sequence of length 3, say `AGU`, the hash table stores the probability of that codon among codons encoding the same amino acid.
Put your solution in the ```get_probabability_dict``` function.

In [236]:

def get_probabability_dict():
    try:
        with open("src/Codon usage table.html") as f:
            soup = BeautifulSoup(f)
    except:
        try:
            with open("Codon usage table.html") as f:
                soup = BeautifulSoup(f)
        except:
            print("Something went wrong")
    
            #   ['UUU', 'F', '714298']
            #    UUU                F                    714298
    regex = r"([A-Z]{3})\s+([A-Z]{1}|[*]{1}).{11,13}\(\s?(\d+)"  # regex magic!
    
    
    data = str(soup.pre).replace("<pre>", "").replace("</pre>", "")
    fdata = re.findall(regex, data)
    

    
    
    codon_freq_list = defaultdict(list)
    for codon, amino, freq in fdata:
        codon_freq_list[codon] = int(freq)
    
    amino_freq_list = defaultdict(list)
    for codon, amino, freq in fdata:
        amino_freq_list[amino].append(int(freq))    
    
    freq_list_sum = defaultdict(list)
    for amino, freqs in amino_freq_list.items():
        freq_list_sum[amino] = sum(freqs)
    
    
    
    codon_to_amino = get_dict()

    
    
    prob_list = defaultdict(list)
    for codon, freq in codon_freq_list.items():
        amino = codon_to_amino[codon]
        summ = freq_list_sum[amino[0]]
        if summ == 0:
            summ = 1
        prob_list[codon] = freq / summ
        
        
#     return lof(fdata)   
    return dict(prob_list)    
    
def lof(a):
    return len(a)
    
if __name__ == '__main__':
    """
    codon_to_prob = get_probabability_dict()   #  PROBABABILITY, REALLY?
    #codon_to_prob = get_probability_dict()
    items = sorted(codon_to_prob.items(), key=lambda x: x[0])
    for i in range(1 + len(items)//6):
        print("\t".join(
            f"{k}: {v:.6f}"
            for k, v in items[i*6:6+i*6]
        ))
    """

    print(get_probabability_dict())    

{'AAA': 0.43404935110207155,
 'AAC': 0.5296330092531971,
 'AAG': 0.5659506488979285,
 'AAU': 0.4703669907468028,
 'ACA': 0.28418772983891855,
 'ACC': 0.35523154074391966,
 'ACG': 0.11381188540055791,
 'ACU': 0.2467688440166039,
 'AGA': 0.21465774794262568,
 'AGC': 0.23993794345056002,
 'AGG': 0.21109127083138968,
 'AGU': 0.14960182300967595,
 'AUA': 0.16906151748761036,
 'AUC': 0.4698662895003286,
 'AUG': 1.0,
 'AUU': 0.36107219301206106,
 'CAA': 0.26501675921017337,
 'CAC': 0.5814847871566309,
 'CAG': 0.7349832407898266,
 'CAU': 0.4185152128433691,
 'CCA': 0.2766025276376192,
 'CCC': 0.3234703981288551,
 'CCG': 0.11319574457649788,
 'CCU': 0.2867313296570278,
 'CGA': 0.1088124833207855,
 'CGC': 0.18377662978978224,
 'CGG': 0.20155434006721587,
 'CGU': 0.08010752804820104,
 'CUA': 0.0713801723134756,
 'CUC': 0.1955768259144855,
 'CUG': 0.39570159041596514,
 'CUU': 0.13171591206484023,
 'GAA': 0.42245266280361615,
 'GAC': 0.5354575808069573,
 'GAG': 0.5775473371963838,
 'GAU': 0.4645424

### Idea of solution

Regex to get the numbers. My solution has a lot of helper dictionaries, but it seems clearest to me that way. 

First helper is codon_freq_list which just stores the codon's number. 
Second helper is amino_freq_list which lists all codon numbers for one aminoacid.
Third is freq_list_sum which sums all codon numbers for one aminoacid.

Then dict prob_list takes each codon, and divides its number by sum of all codon numbers in that aminoacid. There we have the probability.

### Discussion

If I were to load the data into pandas dataframes, I could perhaps get the sum for an aminoacid faster by groupby() function... Though a whole new set of problems arises when going this route.

Now you should have everything in place to easily solve the following.


6. Write a class ```ProteinToMaxRNA``` with a ```convert``` method which converts a protein sequence into the most likely RNA sequence to be the source of this protein. Run your program with `LTPIQNRA`.

In [143]:
class ProteinToMaxRNA:
    
    def __init__(self):
        self.prob_dict = get_probabability_dict()
        self.amino_to_codon = get_dict_list()
        
        
    def convert(self, s):
        liste = []
        
        for i in s:
            codon_list = self.amino_to_codon.get(i)
            #codon_list = get_dict_list().get(i)
            
            most_prob_codon = ""
            most_prob = 0
            for j in codon_list:
                prob = self.prob_dict.get(j)
                #prob = get_probabability_dict().get(j)
                
                if prob > most_prob:
                    most_prob = prob
                    most_prob_codon = j
            
            liste.append(most_prob_codon)
        
        return ''.join(i for i in liste)


if __name__ == '__main__':
    protein_to_rna = ProteinToMaxRNA()
    print(protein_to_rna.convert("LTPIQNRA"))

'CUGACCCCCAUCCAGAACAGAGCC'


### Idea of solution

Method converts each aminoacid to a list of codons. Assigns probabilities to each, and chooses the one with the highest probability. Could probably do this in a less <i>"cody"</i> type of way, with max(), but this clearly demonstrates what actually happens when comparing probabilities.

### Discussion

Bit clueless about what to put in \__init__ method.. I reference the other get_dict methods several times anyway..

<br></br>



Now we are almost ready to produce random RNA sequences that code a given protein sequence. For this, we need a subroutine to *sample from a probability distribution*. Consider our earlier example of probabilities 36/100, 47/100, and 17/100 for `AUU`, `AUC`, and `AUA`, respectively. 
Let us assume we have a random number generator ```random()``` that returns a random number from interval $[0,1)$. We may then partition the unit interval according to cumulative probabilities to $[0,36/100), [36/100,83/100), [83/100,1)$, respectively. Depending which interval the number ```random()``` hits, we select the codon accordingly.

7. Write a function ```random_event``` that chooses a random event, given a probability distribution (set of events whose probabilities sum to 1).
You can use function ```random.uniform``` to produce values uniformly at random from the range $[0,1)$. The distribution should be given to your function as a dictionary from events to their probabilities.

In [186]:
def random_event(dist):
    """
    Takes as input a dictionary from events to their probabilities.
    Return a random event sampled according to the given distribution.
    The probabilities must sum to 1.0
    """
    val = list(dist.values())
    key = list(dist.keys())
    
    
    return random.choice(key, p = val)

    #return next(iter(dist))

if __name__ == '__main__':
    distribution = dict(zip("ACGT", [0.10, 0.35, 0.15, 0.40]))
    print(", ".join(random_event(distribution) for _ in range(29)))
    #print(random_event(distribution))

('A, G, C, T, T, C, C, A, T, C, C, A, C, A, C, T, T, T, C, G, C, C, C, G, T, '
 'T, T, G, T')


### Idea of solution

Numpy random has quite a few useful features. np.random.uniform could be used, but choice is what I've chosen. It choosen a random event from a sequence (numbers, or in this case nucleotids) based on a cumulative distribution function.


### Discussion

fill in 

With this general routine, the following should be easy to solve.
 
8. Write a class ```ProteinToRandomRNA``` to produce a random RNA sequence encoding the input protein sequence according to the input codon adaptation probabilities. The actual conversion is done through the ```convert``` method. Run your program with `LTPIQNRA`.

In [237]:


class ProteinToRandomRNA(object):
    
    def __init__(self):
        prob_dict = get_probabability_dict()
        amino_to_codon = get_dict_list()
        
    def convert(self, s):
        liste = []
        
        for i in s:
            codon_list = get_dict_list().get(i)
            rnd_codon = np.random.choice(codon_list)
            liste.append(rnd_codon)
        
        return ''.join(i for i in liste)
        

if __name__ == '__main__':
   
    protein_to_random_codons = ProteinToRandomRNA()
    print(protein_to_random_codons.convert("LTPIQNRA"))


'CUUACGCCGAUCCAGAAUCGCGCG'


### Idea of solution

Instead of choosing based on a given weighted distribution, now choose randomly from a codon-list. 

### Discussion

fill in 

## Generating DNA sequences with higher-order Markov chains

We will now reuse the machinery derived above in a related context. We go back to DNA sequences, and consider some easy statistics that can be used to characterize the sequences. 
First, just the frequencies of bases $\texttt{A}$, $\texttt{C}$, $\texttt{G}$, $\texttt{T}$ may reveal the species from which the input DNA originates; each species has a different base composition that has been formed during evolution. 
More interestingly, the areas where DNA to RNA transcription takes place (coding region) have an excess of $\texttt{C}$ and $\texttt{G}$ over $\texttt{A}$ and $\texttt{T}$. To detect such areas a common routine is to just use a *sliding window* of fixed size, say $k$, and compute for each window position 
$T[i..i+k-1]$ the base frequencies, where $T[1..n]$ is the input DNA sequence. When sliding the window from  $T[i..i+k-1]$ to $T[i+1..i+k]$ frequency $f(T[i])$ gets decreases by one and $f(T[i+k])$ gets increased by one. 

9. Write a *generator* ```sliding_window``` to compute sliding window base frequencies so that each moving of the window takes constant time. We saw in the beginning of the course one way how to create generators using
  generator expression. Here we use a different way. For the function ```sliding_window``` to be a generator, it must have at least   one ```yield``` expression, see [https://docs.python.org/3/reference/expressions.html#yieldexpr](https://docs.python.org/3/reference/expressions.html#yieldexpr).
  
  Here is an example of a generator expression that works similarily to the built in `range` generator:
  ```Python
  def range(a, b=None, c=1):
      current = 0 if b == None else a
      end = a if b == None else b
      while current < end:
          yield current
          current += c
  ```
  A yield expression can be used to return a value and *temporarily* return from the function.

In [238]:
def sliding_window(s, k):
    """
    This function returns a generator that can be iterated over all
    starting position of a k-window in the sequence.
    For each starting position the generator returns the nucleotide frequencies
    in the window as a dictionary.
    """
    # (n - k + 1) chunks in input string

    nucleotides = ["A", "C", "G", "T"]
    
    for i in range(0, len(s) - k + 1):
        chunk = s[i: i+k]
        freq = {str(letter) : chunk.count(letter) for letter in nucleotides}
        yield freq
        
        
    
if __name__ == '__main__':
    s = "TCCCGACGGCCTTGCC"
    for d in sliding_window(s, 4):
        print(d)

{'A': 0, 'C': 3, 'G': 0, 'T': 1}
{'A': 0, 'C': 3, 'G': 1, 'T': 0}
{'A': 1, 'C': 2, 'G': 1, 'T': 0}
{'A': 1, 'C': 2, 'G': 1, 'T': 0}
{'A': 1, 'C': 1, 'G': 2, 'T': 0}
{'A': 1, 'C': 1, 'G': 2, 'T': 0}
{'A': 0, 'C': 2, 'G': 2, 'T': 0}
{'A': 0, 'C': 2, 'G': 2, 'T': 0}
{'A': 0, 'C': 2, 'G': 1, 'T': 1}
{'A': 0, 'C': 2, 'G': 0, 'T': 2}
{'A': 0, 'C': 1, 'G': 1, 'T': 2}
{'A': 0, 'C': 1, 'G': 1, 'T': 2}
{'A': 0, 'C': 2, 'G': 1, 'T': 1}


### Idea of solution

Iterates through string in k-length parts, and count the nucleotides in them. Dict comprehension {} is really useful here.

### Discussion

Thought the solution first should return the actual chunks (kmers), which in a way was just as functional moving forward. The counts (frequencies) were easier to use though.

 
Our models so far have been so-called *zero-order* models, as each event has been independent of other events. With sequences, the dependencies of events are naturally encoded by their *contexts*. Considering that a sequence is produced from left-to-right, a *first-order* context for $T[i]$ is $T[i-1]$, that is, the immediately preceding symbol. *First-order Markov chain* is a sequence produced by generating $c=T[i]$ with the probability of event of seeing symbol $c$ after previously generated symbol $a=T[i-1]$. The first symbol of the chain is sampled according to the zero-order model.  
The first-order model can naturally be extended to contexts of length $k$, with $T[i]$ depending on $T[i-k..i-1]$. Then the first $k$ symbols of the chain are sampled according to the zero-order model.  The following assignments develop the routines to work with the *higher-order Markov chains*. 
In what follows, a $k$-mer is a substring $T[i..i+k-1]$ of the sequence at an arbitrary position. 



10. Write function ```context_list``` that given an input DNA sequence $T$ associates to each $k$-mer $W$ the concatenation of all symbols $c$ that appear after context $W$ in $T$, that is, $T[i..i+k]=Wc$. For example, <span style="color:red; font:courier;">GA</span> is associated to <span style="color:blue; font: courier;">TCT</span> in $T$=<span style="font: courier;">AT<span style="color:red;">GA</span><span style="color:blue;">T</span>ATCATC<span style="color:red;">GA</span><span style="color:blue;">C</span><span style="color:red;">GA</span><span style="color:blue;">T</span>GTAG</span>, when $k=2$.

In [239]:
def k_generator(s, k):
    
    for i in range(0, len(s) - k):
        chunk = s[i: i+k]
        
#         if i == len(s) - k:    # added if statement for last k-mer that doesn't have a context.
#             next_chunk = ''    # can't do that, breaks context_probababilities..
#         else:
        next_chunk = s[i + k]
            
        yield chunk, next_chunk

def context_list(s, k):
    d = defaultdict(str)
    
    for chunk, next_chunk in k_generator(s, k):
        d[chunk] = d[chunk] + next_chunk
    
    return dict(d)

if __name__ == '__main__':
    k = 2
    s = "ATGATATCATCGACGATCTAG"
    ss = "ABCDA"

    d = context_list(s, k)
    print(d)

    

{'AC': 'G',
 'AT': 'GACCC',
 'CA': 'T',
 'CG': 'AA',
 'CT': 'A',
 'GA': 'TCT',
 'TA': 'TG',
 'TC': 'AGT',
 'TG': 'A'}


### Idea of solution

K_generator helper function returns a k-sized chunk from string s, and also the next k-sized chunk. Then context_list iterates through them and appends the next_chunk (context) to the original k-mer.

### Discussion

First did this with a classic for loop, and without the generator helper function. Worked fine, but had problems with k = 0 and k = 1. Made exceptions for these in the next assignment, but realized it probably would function better if `context_list` had a generator function to help. 

<br></br>

11. With the above solution, write function ```context_probabilities``` to count the frequencies of symbols in each context and convert these frequencies into probabilities. Run `context_probabilities` with $T=$ `ATGATATCATCGACGATGTAG` and $k$ values 0 and 2.

In [241]:

nucleotides = ["A", "C", "G", "T"]

def context_probabilities(s, k):
    d = defaultdict(list)
    for kmer, context in context_list(s, k).items():
        d[kmer] = {str(nuc): context.count(nuc) / len(context) for nuc in nucleotides}
        
    return dict(d)

if __name__ == '__main__':
    s = "ATGATATCATCGACGATGTAG"
    
    # output with k = 0: {'': {'A': 0.333333, 'C': 0.142857, 'T': 0.285714, 'G': 0.238095}}
    
    print(context_probabilities(s, 0))
    
    print(context_probabilities(s, 2))



{'': {'A': 0.3333333333333333,
      'C': 0.14285714285714285,
      'G': 0.23809523809523808,
      'T': 0.2857142857142857}}
{'AC': {'A': 0.0, 'C': 0.0, 'G': 1.0, 'T': 0.0},
 'AT': {'A': 0.2, 'C': 0.4, 'G': 0.4, 'T': 0.0},
 'CA': {'A': 0.0, 'C': 0.0, 'G': 0.0, 'T': 1.0},
 'CG': {'A': 1.0, 'C': 0.0, 'G': 0.0, 'T': 0.0},
 'GA': {'A': 0.0, 'C': 0.3333333333333333, 'G': 0.0, 'T': 0.6666666666666666},
 'GT': {'A': 1.0, 'C': 0.0, 'G': 0.0, 'T': 0.0},
 'TA': {'A': 0.0, 'C': 0.0, 'G': 0.5, 'T': 0.5},
 'TC': {'A': 0.5, 'C': 0.0, 'G': 0.5, 'T': 0.0},
 'TG': {'A': 0.5, 'C': 0.0, 'G': 0.0, 'T': 0.5}}


### Idea of solution

Iterate through all possible nucleotides and count their probabilities. Probabilities counted by dividing count of nucleotide in context by the length of the context itself. 


### Discussion

Previously implemented special k = 0 case in this method, but found a better way to implement it into context_list with generator functions.

12. With the above solution and the function ```random_event``` from the earlier exercise, write class ```MarkovChain```. Its ```generate``` method should generate a random DNA sequence following the original $k$-th order Markov chain probabilities. 

In [248]:


class MarkovChain(object):

    def __init__(self, zeroth, kth, k = 2):
        self.k = k
        self.zeroth = zeroth
        self.kth = kth

    def generate(self, n, seed=None):
        random.seed(seed) 
        
        # if n is 0 or 1, return '': A,C,G,T
        if(self.k > n):
            return random_event(zeroth) * n
        
        #need a first random kmer to start generating random ones.
        
        seq = np.random.choice(list(self.kth.keys()))
        
        for i in range(0, n - self.k):
            prev_kmer = seq[i: i + self.k]
            
            sample_nuc = random_event(self.kth[prev_kmer])
            seq = seq + sample_nuc
        
        return seq


if __name__ == '__main__':
    zeroth = {'A': 0.2, 'C': 0.19, 'T': 0.31, 'G': 0.3}
    kth = {'GT': {'A': 1.0, 'C': 0.0, 'T': 0.0, 'G': 0.0},
           'CA': {'A': 0.0, 'C': 0.0, 'T': 1.0, 'G': 0.0},
           'TC': {'A': 0.5, 'C': 0.0, 'T': 0.0, 'G': 0.5},
           'GA': {'A': 0.0, 'C': 0.3333333333333333, 'T': 0.6666666666666666, 'G': 0.0},
           'TG': {'A': 0.5, 'C': 0.0, 'T': 0.5, 'G': 0.0},
           'AT': {'A': 0.2, 'C': 0.4, 'T': 0.0, 'G': 0.4},
           'TA': {'A': 0.0, 'C': 0.0, 'T': 0.5, 'G': 0.5},
           'AC': {'A': 0.0, 'C': 0.0, 'T': 0.0, 'G': 1.0},
           'CG': {'A': 1.0, 'C': 0.0, 'T': 0.0, 'G': 0.0}
          }
    n = 10
    seed =4
    mc = MarkovChain(zeroth, kth)
    print(mc.generate(n, seed))

'ACGATCATGT'


### Idea of solution

If the k-size is larger than n (size of to-be generated string), return zeroth probabilities of each nucleotide.

Otherwise picks a random kmer from kth dict.keys, then based on that kmers nucleotides probabilities picks the next one, and concenates into string.

### Discussion

seed(0) gives keyError: 'AG' and I haven't really figured out why. Class seems to work as it should otherwise. If seed is removed alltogether, commented out, desired output it reached after a few tries.

If you have survived so far without problems, please run your program a few more times with different inputs. At some point you should get a lookup error in your hash-table! The reason for this is not your code, but the way we defined the model: Some $k$-mers may not be among the training data (input sequence $T$), but such can be generated as the first $k$-mer that is generated using the zero-order model.  

A general approach to fixing such issues with incomplete training data is to use *pseudo counts*. That is, all imaginable events are initialized to frequency count 1.   

13. Write a new solution `context_pseudo_probabilities` based on the solution to problem 11. But this time use pseudo counts in order to obtain a $k$-th order Markov chain that can assign a probability for any DNA sequence. You may use the standard library function `itertools.product` to iterate over all $k$-mer of given length (`product("ACGT", repeat=k)`).

In [180]:
nucleotides = ["A", "C", "G", "T"]

def pseudo_probabilities(s):
    # natural probabilities, now we need them to sum to 1
    #return {str(nuc): s.count(nuc) / len(s) for nuc in nucleotides}
    
    return {str(nuc): (s.count(nuc) + 1) / (len(s) + len(nucleotides)) for nuc in nucleotides}

def context_pseudo_probabilities(s, k):
    d = {}
    
    if k == 0:
        return {"" : pseudo_probabilities(s)}
    
    # iterating through all possible kmers in given string
    for i in itertools.product("ACGT", repeat = k):
        kmer = ''.join(i)
        
        #if given kmer is found in our string, search its context
        c = context_list(s, k)
        
        if kmer in s:
            try:
                context = c[kmer]
            except:
                pass #keyerror ('AG') pops up. invalid really
            
            d[kmer] = pseudo_probabilities(context)
        #if given kmer is not found in string, assign equal pseudoprobabilities to each nucleotide
        else: 
            d[kmer] = {str(nuc): 0.25 for nuc in nucleotides}
        
    return d
    
if __name__ == '__main__':
    #s = "ATGATATCATCGACGATGTAG"
    #print(context_pseudo_probabilities(s, 2))
    #print(context_list(s,2))

    
    
    k = 2
    s = "ATGATATCATCGACGATGTAG"
    kth = context_pseudo_probabilities(s, k)
    zeroth = context_pseudo_probabilities(s, 0)
    print(f"zeroth: {zeroth}")
    #print("".join(f"{k}: {dict(v)}" for k, v in kth.items()))
    print(kth)
    
    print(MarkovChain(zeroth, kth, k).generate(20, 0))


"zeroth: {'': {'A': 0.32, 'C': 0.16, 'G': 0.24, 'T': 0.28}}"
{'AA': {'A': 0.25, 'C': 0.25, 'G': 0.25, 'T': 0.25},
 'AC': {'A': 0.2, 'C': 0.2, 'G': 0.4, 'T': 0.2},
 'AG': {'A': 0.2, 'C': 0.2, 'G': 0.4, 'T': 0.2},
 'AT': {'A': 0.2222222222222222,
        'C': 0.3333333333333333,
        'G': 0.3333333333333333,
        'T': 0.1111111111111111},
 'CA': {'A': 0.2, 'C': 0.2, 'G': 0.2, 'T': 0.4},
 'CC': {'A': 0.25, 'C': 0.25, 'G': 0.25, 'T': 0.25},
 'CG': {'A': 0.5,
        'C': 0.16666666666666666,
        'G': 0.16666666666666666,
        'T': 0.16666666666666666},
 'CT': {'A': 0.25, 'C': 0.25, 'G': 0.25, 'T': 0.25},
 'GA': {'A': 0.14285714285714285,
        'C': 0.2857142857142857,
        'G': 0.14285714285714285,
        'T': 0.42857142857142855},
 'GC': {'A': 0.25, 'C': 0.25, 'G': 0.25, 'T': 0.25},
 'GG': {'A': 0.25, 'C': 0.25, 'G': 0.25, 'T': 0.25},
 'GT': {'A': 0.4, 'C': 0.2, 'G': 0.2, 'T': 0.2},
 'TA': {'A': 0.16666666666666666,
        'C': 0.16666666666666666,
        'G': 0.33333

### Idea of solution

Pseudo_probabilities function is used to assert probabilities to nucleotides that sum up to one. 

Then basically the same solution, except if the kmer isn't found in input string, give the nucleotides equal probabilities each.

### Discussion

Messy print right now, have to format/shorten the decimals.

<!-- removed the [""] from <i>zeroth = context_pseudo_probabilities(s, 0) </i>, dict messes up somehow...,  -->



14. Write class ```MarkovProb``` that given the $k$-th order Markov chain developed above to the constructor, its method ```probability``` computes the probability of a given input DNA sequence.

In [250]:
class MarkovProb:
    
    def __init__(self, k, zeroth, kth):
        self.k = k
        self.zeroth = zeroth
        self.kth = kth
        
    def k_next_nuc_generator(self, s):
        
        """for self.k sized chunk, yield chunk and next nucleotide. """
      
        for i in range(len(s) - self.k):
            kmer = s[i: i + self.k]
            next_n = s[i + self.k]
            yield (kmer, next_n)
            
    def probability(self, s):
        # multiply all nucleotides inside 
        
        zeroth_prob_array = [self.zeroth[nuc] for nuc in s[: self.k]]
        prob = np.product(zeroth_prob_array)
        
        #return prob
        
        for kmer, next_nuc in self.k_next_nuc_generator(s):
            next_prob = self.kth[kmer][next_nuc] # what was the probability of the next nuc being what it was?
            prob = prob * next_prob              # adding that to the total

        return prob
    
    
    
if __name__ == '__main__':
    
    
    k = 2
    kth = context_pseudo_probabilities("ATGATATCATCGACGATGTAG", k)
    zeroth = context_pseudo_probabilities("ATGATATCATCGACGATGTAG", 0)[""]
    mc = MarkovProb(2, zeroth, kth)
    s="ATGATATCATCGACGATGTAG"
    
    print(mc.probability(s))

    
    print(f"Probability of sequence {s} is {mc.probability(s)}")
    
    

2.831270190340017e-10
'Probability of sequence ATGATATCATCGACGATGTAG is 2.831270190340017e-10'


### Idea of solution

The initial probabilities need to be multiplied from the nucleotides (obtained from the self.zeroth dict). Then we get the current kmer and the next nucleotide from the generator function, and retrieve the probabilities for these from the self.kth probability dict. These multiplied together are then returned.


### Discussion

fill in

With the last assignment you might end up in trouble with precision, as multiplying many small probabilities gives a really small number in the end. There is an easy fix by using so-called log-transform. 
Consider computation of $P=s_1 s_2 \cdots s_n$, where $0\leq s_i\leq 1$ for each $i$. Taking logarithm in base 2 from both sides gives $\log _2 P= \log _2 (s_1 s_2 \cdots s_n)=\log_2 s_1 + \log_2 s_2 + \cdots \log s_n= \sum_{i=1}^n \log s_i$, with repeated application of the property that the logarithm of a multiplication of two numbers is the sum of logarithms of the two numbers taken separately. The results is abbreviated as log-probability.

15. Write class ```MarkovLog``` that given the $k$-th order Markov chain developed above to the constructor, its method ```log_probability``` computes the log-probability of a given input DNA sequence. Run your program with $T=$ `ATGATATCATCGACGATGTAG` and $k=2$.

In [251]:
class MarkovLog(object):

    def __init__(self, k, zeroth, kth):
        self.k = k
        self.zeroth = {nucleotide: log2(prob) for nucleotide, prob in zeroth.items()}
        self.kth = {kmer : {n : log2(p) for n, p in prob_list.items() } for kmer, prob_list in kth.items()  }
        
    def k_next_nuc_generator(self, s):
        
        __doc__ = """for self.k sized chunk, yield chunk and next nucleotide. """
        i = 0
        
        while i < len(s) - self.k:
            kmer = s[i : i +self.k]
            next_n = s[i + self.k]
            i += 1
            yield (kmer, next_n)

    def log_probability(self, s):
        
        # sum!  all nucleotides inside 
        
        zeroth_prob_array = [self.zeroth[nuc] for nuc in s[: self.k]]
        prob = np.sum(zeroth_prob_array)
        
        #return prob
        
        for kmer, next_nuc in self.k_next_nuc_generator(s):
            next_prob = self.kth[kmer][next_nuc]
            prob = prob + next_prob

        return prob
    
    
    
if __name__ == '__main__':
    k = 2
    kth = context_pseudo_probabilities("ATGATATCATCGACGATGTAG", k)
    zeroth = context_pseudo_probabilities("ATGATATCATCGACGATGTAG", 0)[""]
    mc = MarkovLog(2, zeroth, kth)
    s="ATGATATCATCGACGATGTAG"
    print(f"Log probability of sequence {s} is {mc.log_probability(s)}")

'Log probability of sequence ATGATATCATCGACGATGTAG is -31.717831515538315'


### Idea of solution

Basically same as before but with constructor probabilities transformed to log2s. Also final probability is summed rather than multiplied.

### Discussion

Have to check if it works proberly. log<sub>2</sub>(n) should == $2^n$	which it does: 

n = -31.717831515538315 (from "ATGATATCATCGACGATGTAG") = 2.831270190340017e-10, same as previous exercise!




Finally, if you try to use the code so far for very large inputs, you might observe that the concatenation of symbols following a context occupy considerable amount of space. This is unnecessary, as we only need the frequencies. 

16. Optimize the space requirement of your code from exercise 13 for the $k$-th order Markov chain by replacing the concatenations by direct computations of the frequencies. Implement this as the
  ```better_context_probabilities``` function.

In [220]:

nucleotides = ["A", "C", "G", "T"]

def better_context_probabilities(s, k):
    d = {}
    dd = {}
    d3 = {}
    if k == 0:
        return pseudo_probababilities(s)
    
#     iterating through all possible kmers in given string
    for i in itertools.product("ACGT", repeat = k):
        kmer = ''.join(i)
        
            
         
        d[kmer] = {str(nuc): 0.25 for nuc in nucleotides}
    return d


if __name__ == '__main__':
    k = 2
    s = "ATGATATCATCGACGATGTAG"
    d = better_context_probabilities(s, k)
    #print("\n".join(f"{k}: {v}" for k, v in d.items()))
    print(d)

{'AA': {'A': 0.25, 'C': 0.25, 'G': 0.25, 'T': 0.25},
 'AC': {'A': 0.25, 'C': 0.25, 'G': 0.25, 'T': 0.25},
 'AG': {'A': 0.25, 'C': 0.25, 'G': 0.25, 'T': 0.25},
 'AT': {'A': 0.25, 'C': 0.25, 'G': 0.25, 'T': 0.25},
 'CA': {'A': 0.25, 'C': 0.25, 'G': 0.25, 'T': 0.25},
 'CC': {'A': 0.25, 'C': 0.25, 'G': 0.25, 'T': 0.25},
 'CG': {'A': 0.25, 'C': 0.25, 'G': 0.25, 'T': 0.25},
 'CT': {'A': 0.25, 'C': 0.25, 'G': 0.25, 'T': 0.25},
 'GA': {'A': 0.25, 'C': 0.25, 'G': 0.25, 'T': 0.25},
 'GC': {'A': 0.25, 'C': 0.25, 'G': 0.25, 'T': 0.25},
 'GG': {'A': 0.25, 'C': 0.25, 'G': 0.25, 'T': 0.25},
 'GT': {'A': 0.25, 'C': 0.25, 'G': 0.25, 'T': 0.25},
 'TA': {'A': 0.25, 'C': 0.25, 'G': 0.25, 'T': 0.25},
 'TC': {'A': 0.25, 'C': 0.25, 'G': 0.25, 'T': 0.25},
 'TG': {'A': 0.25, 'C': 0.25, 'G': 0.25, 'T': 0.25},
 'TT': {'A': 0.25, 'C': 0.25, 'G': 0.25, 'T': 0.25}}


### Idea of solution

fill in

### Discussion

fill in

While the earlier approach of explicit concatenation of symbols following a context suffered from inefficient use of space, it does have a benefit of giving another much simpler strategy to sample from the distribution: 
observe that an element of the concatenation taken uniformly randomly is sampled exactly with the correct probability. 

17. Revisit the solution 12 and modify it to directly sample from the concatenation of symbols following a context. The function ```np.random.choice``` may be convenient here. Implement the modified version as the new `SimpleMarkovChain` class.

In [18]:
class SimpleMarkovChain(object):
    def __init__(self, s, k):
        pass

    def generate(self, n, seed=None):
        return "🐍"*n
        
if __name__ == '__main__':
    k = 2
    s = "ATGATATCATCGACGATGTAG"
    n = 10
    seed = 7
    mc = SimpleMarkovChain(s, k)
    print(mc.generate(n, seed))

🐍🐍🐍🐍🐍🐍🐍🐍🐍🐍


### Idea of solution

fill in

### Discussion

fill in

## $k$-mer index

Our $k$-th order Markov chain can now be modified to a handy index structure called $k$-mer index. This index structure associates to each $k$-mer its list of occurrence positions in DNA sequence $T$.  Given a query $k$-mer $W$, one can thus easily list all positions $i$ with  $T[i..k-1]=W$.

18. Implement function ```kmer_index``` inspired by your earlier code for the $k$-th order Markov chain. Test your program with `ATGATATCATCGACGATGTAG` and $k=2$.

In [183]:

def kmer_index_generator(s, k):    
    
    for i in range(0, len(s) - k + 1):
        chunk = s[i: i+k]
        yield (chunk, i)
        

def kmer_index(s, k):
    d = defaultdict(list)
    
    for k, i in kmer_index_generator(s, k):
        d[k].append(i)
        
    return dict(d)

if __name__ == '__main__':
    k=2
    s = "ATGATATCATCGACGATGTAG"
    print("Using string:")
    print(s)
    print("".join([str(i%10) for i in range(len(s))]))
    print(f"{k}-mer index is:")
    d=kmer_index(s, k)
    print(dict(d))

'Using string:'
'ATGATATCATCGACGATGTAG'
'012345678901234567890'
'2-mer index is:'
{'AC': [12],
 'AG': [19],
 'AT': [0, 3, 5, 8, 15],
 'CA': [7],
 'CG': [10, 13],
 'GA': [2, 11, 14],
 'GT': [17],
 'TA': [4, 18],
 'TC': [6, 9],
 'TG': [1, 16]}


### Idea of solution

Generator functions are useful, used one to yield every k-sized kmer and it's index. Then iterate through all of them and append to a defaultdict(list) so that they're grouped together. 

### Discussion

Should the indexing be in form "012345678901234567890" instead as example should suggest? Probably not, would be much harder to actually use the indexes in an even slightly longer sequence.

## Comparison of probability distributions

Now that we know how to learn probability distributions from data, we might want to compare two such distributions, for example, to test if our programs work as intended. 

Let $P=\{p_1,p_2,\ldots, p_n\}$ and $Q=\{q_1,q_2,\ldots, q_n\}$ be two probability distributions for the same set of $n$ events. This means $\sum_{i=1}^n p_i=\sum_{i=1}^n q_i=1$, $0\leq p_j \leq 1$, and $0\leq q_j \leq 1$ for each event $j$. 

*Kullback-Leibler divergence* is a measure $d()$ for the *relative entropy* of $P$ with respect to $Q$ defined as 
$d(P||Q)=\sum_{i=1}^n p_i \log\frac{p_i}{q_i}$.


This measure is always non-negative, and 0 only when $P=Q$. It can be interpreted as the gain of knowing $Q$ to encode $P$. Note that this measure is not symmetric.

19. Write function ```kullback_leibler``` to compute $d(P||Q)$. Test your solution by generating a random RNA sequence
  encoding the input protein sequence according to the input codon adaptation probabilities.
  Then you should learn the codon adaptation probabilities from the RNA sequence you generated.
  Then try the same with uniformly random RNA sequences (which don't have to encode any
  specific protein sequence). Compute the relative entropies between the
  three distribution (original, predicted, uniform) and you should observe a clear difference.
  Because $d(P||Q)$ is not symmetric, you can either print both $d(P||Q)$ and $d(Q||P)$,
  or their average.
  
  This problem may be fairly tricky. Only the `kullback_leibler` function is automatically tested. The codon probabilities is probably a useful helper function. The main guarded section can be completed by filling out the `pass` sections using tooling from previous parts and fixing the *placeholder* lines.

In [229]:
def codon_probabilities(rna):
    """
    Given an RNA sequence, simply calculates the proability of
    all 3-mers empirically based on the sequence
    """
    return {"".join(codon): 0 for codon in np.product("ACGU", repeat=3)}
    
def kullback_leibler(p, q):
    """
    Computes Kullback-Leibler divergence between two distributions.
    Both p and q must be dictionaries from events to probabilities.
    The divergence is defined only when q[event] == 0 implies p[event] == 0.
    """
    return sum(p[i] * log2(p[i]/q[i]) for i in range(len(p)))

if __name__ == '__main__':
    aas = list("*ACDEFGHIKLMNPQRSTVWY") # List of amino acids
    n = 10000
    
    # generate a random protein and some associated rna
    protein = "".join(random.choice(aas, n))    
    pass
    
    # Maybe check that converting back to protein results in the same sequence
    pass
    
    # Calculate codon probabilities of the rna sequence
#     cp_predicted = codon_probabilities("<rna sequence>") # placeholder call
    
    # Calculate codon probabilities based on the codon usage table
    cp_orig = {"".join(codon): 0 for codon in np.product("ACGU", repeat = 3 )} # placeholder dict
    
    # Create a completely random RNA sequence and get the codon probabilities
    pass
    cp_uniform = codon_probabilities("<random rna sequence>") # placeholder call
    
    print("d(original || predicted) =", kullback_leibler(cp_orig, cp_orig))
    print("d(predicted || original) =", kullback_leibler(cp_predicted, cp_orig))
    print()
    print("d(original || uniform) =", kullback_leibler(cp_orig, cp_uniform))
    print("d(uniform || original) =", kullback_leibler(cp_uniform, cp_orig))
    print()
    print("d(predicted || uniform) =", kullback_leibler(cp_predicted, cp_uniform))
    print("d(uniform || predicted) =", kullback_leibler(cp_uniform, cp_predicted))

TypeError: _prod_dispatcher() got an unexpected keyword argument 'repeat'

### Idea of solution

No idea. Did take a look at the actual function of kullback_leibler and implemented it.

### Discussion

fill in

## Stationary and equilibrium distributions (extra)

Let us consider a Markov chain of order one on the set of nucleotides.
Its transition probabilities can be expressed as a $4 \times 4$ matrix
$P=(p_{ij})$, where the element $p_{ij}$ gives the probability of the $j$th nucleotide
on the condition the previous nucleotide was the $i$th. An example of a transition matrix
is

\begin{array}{l|rrrr}
 &     A &    C &     G &    T \\
\hline
A &  0.30 &  0.0 &  0.70 &  0.0 \\
C &  0.00 &  0.4 &  0.00 &  0.6 \\
G &  0.35 &  0.0 &  0.65 &  0.0 \\
T &  0.00 &  0.2 &  0.00 &  0.8 \\
\end{array}.

A distribution $\pi=(\pi_1,\pi_2,\pi_3,\pi_4)$ is called *stationary*, if
$\pi = \pi P$ (the product here is matrix product).

20. Write function ```get_stationary_distributions``` that gets a transition matrix as parameter,
  and returns the list of stationary distributions. You can do this with NumPy by
  first taking transposition of both sides of the above equation to get equation
  $\pi^T = P^T \pi^T$. Using numpy.linalg.eig take all eigenvectors related to
  eigenvalue 1.0. By normalizing these vectors to sum up to one get the stationary distributions
  of the original transition matrix. In the ```main``` function print the stationary distributions
  of the above transition matrix.

In [21]:
def get_stationary_distributions(transition):
    """
    The function get a transition matrix of a degree one Markov chain as parameter.
    It returns a list of stationary distributions, in vector form, for that chain.
    """
    return np.random.rand(2, 4) - 0.5
    
    
if __name__ == "__main__":
    transition=np.array([[0.3, 0, 0.7, 0],
                         [0, 0.4, 0, 0.6],
                         [0.35, 0, 0.65, 0],
                         [0, 0.2, 0, 0.8]])
    print("\n".join(
        ", ".join(
            f"{pv:+.3f}"
            for pv in p) 
        for p in get_stationary_distributions(transition)))

+0.162, -0.227, +0.388, +0.151
-0.370, +0.057, +0.198, +0.037


### Idea of solution


### Discussion


21. Implement the `kl_divergence` function below so that the main guarded code runs properly. Using your modified Markov chain generator generate a nucleotide sequence $s$ of length $10\;000$. Choose prefixes of $s$ of lengths $1, 10, 100, 1000$, and $10\;000$. For each of these prefixes find out their nucleotide distribution (of order 0) using your earlier tool. Use 1 as the pseudo count. Then, for each prefix, compute the KL divergence between the initial distribution and the normalized nucleotide distribution.

In [22]:
def kl_divergences(initial, transition):
    """
    Calculates the the Kullback-Leibler divergences between empirical distributions
    generated using a markov model seeded with an initial distributin and a transition 
    matrix, and the initial distribution.
    Sequences of length [1, 10, 100, 1000, 10000] are generated.
    """
    return zip([1, 10, 100, 1000, 10000], np.random.rand(5))

if __name__ == "__main__":
    transition=np.array([[0.3, 0, 0.7, 0],
                         [0, 0.4, 0, 0.6],
                         [0.35, 0, 0.65, 0],
                         [0, 0.2, 0, 0.8]])
    print("Transition probabilities are:")
    print(transition)
    stationary_distributions = get_stationary_distributions(transition)
    print("Stationary distributions:")
    print(np.stack(stationary_distributions))
    initial = stationary_distributions[1]
    print("Using [{}] as initial distribution\n".format(", ".join(f"{v:.2f}" for v in initial)))
    results = kl_divergences(initial, transition)
    for prefix_length, divergence in results: # iterate on prefix lengths in order (1, 10, 100...)
        print("KL divergence of stationary distribution prefix " \
              "of length {:5d} is {:.8f}".format(prefix_length, divergence))

Transition probabilities are:
[[ 0.3   0.    0.7   0.  ]
 [ 0.    0.4   0.    0.6 ]
 [ 0.35  0.    0.65  0.  ]
 [ 0.    0.2   0.    0.8 ]]
Stationary distributions:
[[ 0.02421173 -0.29026212  0.1389851   0.44623574]
 [ 0.24261234  0.09071543  0.35909807 -0.47662849]]
Using [0.24, 0.09, 0.36, -0.48] as initial distribution

KL divergence of stationary distribution prefix of length     1 is 0.60666836
KL divergence of stationary distribution prefix of length    10 is 0.71775085
KL divergence of stationary distribution prefix of length   100 is 0.38115273
KL divergence of stationary distribution prefix of length  1000 is 0.36116144
KL divergence of stationary distribution prefix of length 10000 is 0.22844669


### Idea of solution

fill in

### Discussion
fill in

22. Implement the following in the ```main``` function.
Find the stationary distribution for the following transition matrix:  

\begin{array}{ l | r r r r}
 & A &     C &     G &     T \\
\hline
A &  0.30 &  0.10 &  0.50 &  0.10 \\
C &  0.20 &  0.30 &  0.15 &  0.35 \\
G &  0.25 &  0.15 &  0.20 &  0.40 \\
T &  0.35 &  0.20 &  0.40 &  0.05 \\
\end{array}

Since there is only one stationary distribution, it is called the *equilibrium distribution*.
Choose randomly two nucleotide distributions. You can take these from your sleeve or
sample them from the Dirichlet distribution. Then for each of these distributions
as the initial distribution of the Markov chain, repeat the above experiment.

The `main` function should return tuples, where the first element is the (random) initial distribution and the second element contains the results as a list of tuples where the first element is the kl divergence and the second element the empirical nucleotide distribution, for the different prefix lengths.

The state distribution should converge to the equilibrium distribution no matter how we
start the Markov chain! That is the last line of the tables should have KL-divergence very close to $0$ and an empirical distribution very close to the equilibrium distribution.


In [23]:
def main(transition, equilibrium_distribution):
    vals = list(zip(np.random.rand(10), np.random.rand(10, 4) - 0.5))
    return zip(np.random.rand(2, 4) - 0.5, 
               [vals[:5], vals[5:]])


if __name__ == "__main__":
    transition = np.array([[0.3, 0.1, 0.5, 0.1],
                           [0.2, 0.3, 0.15, 0.35],
                           [0.25, 0.15, 0.2, 0.4],
                           [0.35, 0.2, 0.4, 0.05]])
    print("Transition probabilities are:", transition, sep="\n")
    stationary_distributions = get_stationary_distributions(transition)
    # Uncomment the below line to check that there actually is only one stationary distribution
    # assert len(stationary_distributions) == 1
    equilibrium_distribution = stationary_distributions[0]
    print("Equilibrium distribution:")
    print(equilibrium_distribution)
    for initial_distribution, results in main(transition, equilibrium_distribution):
        print("\nUsing {} as initial distribution:".format(initial_distribution))
        print("kl-divergence   empirical distribution")
        print("\n".join("{:.11f}   {}".format(di, kl) for di, kl in results))

Transition probabilities are:
[[ 0.3   0.1   0.5   0.1 ]
 [ 0.2   0.3   0.15  0.35]
 [ 0.25  0.15  0.2   0.4 ]
 [ 0.35  0.2   0.4   0.05]]
Equilibrium distribution:
[ 0.05549613 -0.0618828  -0.3045492   0.11913485]

Using [-0.3337754   0.45068768 -0.10300145 -0.20088342] as initial distribution:
kl-divergence   empirical distribution
0.25023050057   [ 0.2243623   0.15184879 -0.36068593  0.0442362 ]
0.85018673369   [-0.27420015 -0.1344076  -0.08342149  0.10517491]
0.69016371310   [-0.11457606  0.27115112  0.16359241 -0.1280264 ]
0.14692236781   [ 0.07706555  0.23608696  0.01647088 -0.08926859]
0.98884289212   [-0.06341222  0.45084425  0.3957673  -0.40553139]

Using [-0.14182502  0.21310283  0.21065106  0.45942999] as initial distribution:
kl-divergence   empirical distribution
0.09700875402   [ 0.35101425 -0.18876535  0.25445247 -0.20776396]
0.51559517427   [ 0.31812104 -0.03010157 -0.2113726  -0.49508434]
0.92023091580   [ 0.40268876 -0.22995087 -0.42421415 -0.1464154 ]
0.32096833450  

### Idea of solution

fill in

### Discussion
fill in